# AI Weather Forecasting using a Long Short-Term Memory (LSTM) Network

**Copyright (c) 2026 Shrikara Kaudambady. All rights reserved.**

This notebook demonstrates how to build and train an LSTM Recurrent Neural Network (RNN) for time-series weather forecasting. We will predict the future temperature based on a sequence of past weather conditions (temperature, humidity, and pressure).

### 1. Setup and Library Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

sns.set_theme(style="whitegrid")

### 2. Data Simulation

We will generate a synthetic multi-variate weather dataset with hourly readings for 3 years. This includes temperature, humidity, and pressure.

In [ ]:
np.random.seed(42)
time_index = pd.date_range(start='2023-01-01', end='2025-12-31 23:00:00', freq='h')
num_hours = len(time_index)

# Temperature: Seasonal + Daily variation + Noise
seasonal_temp = 15 * (1 - np.cos(2 * np.pi * time_index.dayofyear / 365.25))
daily_temp = 5 * (1 - np.cos(2 * np.pi * time_index.hour / 24))
temperature = seasonal_temp + daily_temp + np.random.normal(0, 1.5, num_hours) + 5 # Base temp of 5C

# Humidity: Inversely related to temperature + Noise
humidity = 80 - (temperature * 1.5) + np.random.normal(0, 5, num_hours)
humidity = np.clip(humidity, 20, 100)

# Pressure: Stable with some noise
pressure = 1013 + np.random.normal(0, 2, num_hours)

df = pd.DataFrame({
    'temperature': temperature,
    'humidity': humidity,
    'pressure': pressure
}, index=time_index)

print("Simulated Weather Data:")
df.head()

### 3. Data Preprocessing for LSTM

Neural networks require special data preparation:
1.  **Scaling:** All features must be scaled to a uniform range (e.g., 0 to 1).
2.  **Sequencing:** The data must be converted into windows of sequences (X) and corresponding target values (y).

In [ ]:
# Step 1: Scale the data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# Step 2: Create sequences
# We will use the last 72 hours (3 days) of data to predict the temperature in 12 hours.
N_PAST = 72  # Number of past hours to use as input
N_FUTURE = 12 # Number of hours in the future to predict
N_FEATURES = df.shape[1]

X, y = [], []
for i in range(N_PAST, len(df_scaled) - N_FUTURE + 1):
    X.append(df_scaled[i - N_PAST:i, 0:N_FEATURES])
    y.append(df_scaled[i + N_FUTURE - 1:i + N_FUTURE, 0]) # Predicting temperature

X, y = np.array(X), np.array(y)

# Split into training and testing sets (80/20 split)
split_ratio = 0.8
split_point = int(split_ratio * len(X))
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

### 4. Build and Train the LSTM Model

We will now define our LSTM model architecture using TensorFlow and Keras.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(N_PAST, N_FEATURES), return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training History')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

### 5. Evaluation and Visualization

Let's see how well our model performs by predicting on the test set and comparing the forecast to the actual temperatures.

In [ ]:
# Make predictions on the test set
predictions_scaled = model.predict(X_test)

# Inverse scale the predictions to get actual temperature values
# We need to create a dummy array with the same shape as the original scaler input
dummy_array = np.zeros((len(predictions_scaled), N_FEATURES))
dummy_array[:, 0] = predictions_scaled.flatten()
predictions = scaler.inverse_transform(dummy_array)[:, 0]

# Inverse scale the actual values for comparison
dummy_array_actual = np.zeros((len(y_test), N_FEATURES))
dummy_array_actual[:, 0] = y_test.flatten()
actuals = scaler.inverse_transform(dummy_array_actual)[:, 0]

# Create a DataFrame for plotting
plot_df = pd.DataFrame({
    'Actual Temperature': actuals,
    'Predicted Temperature': predictions
}, index=df.index[split_point + N_PAST + N_FUTURE - 1:])

# Plot the results
plt.figure(figsize=(16, 8))
plot_df['Actual Temperature'].tail(200).plot(label='Actual', lw=2)
plot_df['Predicted Temperature'].tail(200).plot(label='Predicted', lw=2, style='--')
plt.title('Weather Forecast vs. Actual (Last 200 Hours of Test Set)')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.show()